# 필요한 라이브러리 Import

In [1]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model
import random
import numpy as np
from tqdm import tqdm, trange, tqdm_notebook
import pandas as pd
from ast import literal_eval
from torch import nn
from transformers import BertModel, BertConfig
from torchcrf import CRF
from pytorch_transformers import AdamW, WarmupLinearSchedule
from gluonnlp.data import BERTSPTokenizer
import sentencepiece as spm
from seqeval.metrics import f1_score
from pathlib import Path
import json

In [ ]:
class Config:
    def __init__(self, json_path):
        with open(json_path, mode='r') as io:
            params = json.loads(io.read())
        self.__dict__.update(params)

    def save(self, json_path):
        with open(json_path, mode='w') as io:
            json.dump(self.__dict__, io, indent=4)

    def update(self, json_path):
        with open(json_path, mode='r') as io:
            params = json.loads(io.read())
        self.__dict__.update(params)

    @property
    def dict(self):
        return self.__dict__

# KoBERT + CRF

In [ ]:
#클래스 참고: KoBERT와 CRF로 만든 한국어 개체명 인식 https://github.com/eagle705/pytorch-bert-crf-ner

bert_config = {'attention_probs_dropout_prob': 0.1,
                 'hidden_act': 'gelu',
                 'hidden_dropout_prob': 0.1,
                 'hidden_size': 768,
                 'initializer_range': 0.02,
                 'intermediate_size': 3072,
                 'max_position_embeddings': 512,
                 'num_attention_heads': 12,
                 'num_hidden_layers': 12,
                 'type_vocab_size': 2,
                 'vocab_size': 3517,
                 'padding': True,
                 'pair': False
                 }
                 
class KobertCRF(nn.Module):
    """ KoBERT with CRF """
    def __init__(self, config, num_classes, vocab=None) -> None:
        super(KobertCRF, self).__init__()

        if vocab is None:
            self.bert, self.vocab = get_pytorch_kobert_model()
        else:
            self.bert = BertModel(config=BertConfig.from_dict(bert_config))
            self.vocab = vocab

        self.dropout = nn.Dropout(config.dropout)
        self.position_wise_ff = nn.Linear(config.hidden_size, num_classes)
        self.crf = CRF(num_tags=num_classes, batch_first=True)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
        
    def forward(self, input_ids, valid_length, token_type_ids=None, tags=None):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        # outputs: (last_encoder_layer, pooled_output, attention_weight)
        outputs = self.bert(input_ids=input_ids,
                            token_type_ids=token_type_ids,
                            attention_mask=attention_mask.float().to(token_ids.device))
        last_encoder_layer = outputs[0]
        last_encoder_layer = self.dropout(last_encoder_layer)
        emissions = self.position_wise_ff(last_encoder_layer)

        if tags is not None:
            log_likelihood, sequence_of_tags = self.crf(emissions, tags), self.crf.decode(emissions)
            return log_likelihood, sequence_of_tags
        else:
            sequence_of_tags = self.crf.decode(emissions)
            return sequence_of_tags

# Tokenizer & Vocab 생성


In [ ]:
#Tokenizer model 생성

corpus = "vocab.txt" # KoCharELECTRA (https://github.com/monologg/KoCharELECTRA) vocab.txt 파일 사용 (음절 vocab 파일)
prefix = "vocab"
vocab_size = 3511 # vocab.txt 크기

spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=100" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [ ]:
bertmodel, _ = get_pytorch_kobert_model() # KoBERT 모델 불러오기
vocab = nlp.vocab.BERTVocab.from_sentencepiece('vocab.model', padding_token='[PAD]')
tokenizer = "vocab.model" #생성한 vocab.model 이용
tok = BERTSPTokenizer(tokenizer, vocab, lower=False) # BERTSPTokenizer 생성

## 데이터 셋 불러오기

In [2]:
train_list_csv = pd.read_csv('Data/pet_train.csv', converters={"0": literal_eval})
test_list_csv = pd.read_csv('Data/pet_test.csv', converters={"0": literal_eval})
tr_tag = train_list_csv['0']
tr_sent = train_list_csv['1']
ts_tag = test_list_csv['0']
ts_sent = test_list_csv['1']

del_index = 0

for i in range(len(train_list_csv["0"])):
    if len(tr_tag[i]) != len(tr_sent[i])+3: #데이터 셋 준비 과정 중 길이 다르면 해당 인댁스 출력 +3 해주는 이유는 버트에 넣기 전 붙을 태그 등 데이터 떄문
        print(i)
    if len(tr_tag[i]) > 512 or len(tr_sent[i])+3 > 512: # 문장의 길이가 512가 넘는 문장 제거
        del tr_tag[i]
for i in range(len(test_list_csv["0"])):
    if len(ts_tag[i]) != len(ts_sent[i])+3:
        print(i)
    if len(ts_tag[i]) > 512 or len(ts_sent[i])+3 > 512: # 문장의 길이가 512가 넘는 문장 제거
        del tr_sent[i]

## 버트 데이터 셋 클래스 생성

In [ ]:
# 코드 참고: 버트를 이용한 네이버 댓글 분류기 https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, sent, tag, bert_tokenizer, max_len,
                pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)
        self.sentences = [transform([i]) for i in sent] #문장
        self.labels = [np.pad(np.int32(i), (0,max_len-len(i)), 'constant', constant_values=0) for i in tag] #질병 태그 및 질병 태그 max_len 패딩

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## 사용자 입력 문장을 테스하기 위한 데이터 셋 클래스 생성

In [ ]:
class BERTTESTsent(Dataset):
    def __init__(self, sent, bert_tokenizer, max_len,
                pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)
        self.sentences = [transform([i]) for i in sent] #문장

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

## 데이터 셋 나누기

In [ ]:
data_train = BERTDataset(tr_sent,tr_tag, tok, 512, True, False)
data_test = BERTDataset(ts_sent,ts_tag, tok, 512, True, False)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=3, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers = 5)

# 모델 선언 (초기 학습을 위한)

In [ ]:
device = torch.device('cuda')
model_config = Config('config.json')
model = KobertCRF(config=model_config, num_classes=7)
model.to(device)
model.train()

# 학습된 모델 불러오기 (파인 튜닝을 위한)

In [ ]:
model = torch.load('KoBERT_DIS_NER/Human_PET_DIS2.pth')
model_config = Config('config.json')

# 모델 환경 설정

In [ ]:
def set_seed(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    n_gpu = torch.cuda.device_count()
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(
        nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

t_total = len(
    train_dataloader) // model_config.gradient_accumulation_steps * model_config.epochs
optimizer = AdamW(optimizer_grouped_parameters,
                    lr=model_config.learning_rate, eps=model_config.adam_epsilon)
scheduler = WarmupLinearSchedule(
    optimizer, warmup_steps=model_config.warmup_steps, t_total=t_total)

model_dir = 'DIS_NER/'

global_step = 0
tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()
set_seed()

# 모델 학습

In [ ]:
import os

train_iterator = trange(int(model_config.epochs), desc="Epoch")
device = torch.device('cuda')
for _epoch, _ in enumerate(train_iterator):
    epoch = _epoch
    for step, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        
        model.train()
 
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        log_likelihood, sequence_of_tags = model(token_ids,valid_length, segment_ids, label)

        # loss: negative log-likelihood
        loss = -1 * log_likelihood

        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            model.parameters(), model_config.max_grad_norm)
        tr_loss += loss.item()

        if (step + 1) % model_config.gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            global_step += 1

            with torch.no_grad():
                sequence_of_tags = torch.tensor(
                    sequence_of_tags).to(device)
                mb_acc = (sequence_of_tags == label).float()[label != 0].mean()

            tr_acc = mb_acc.item()
            tr_loss_avg = tr_loss / global_step
            tr_summary = {'loss': tr_loss_avg, 'acc': tr_acc}
            state = {'global_step': global_step + 1,
                             'model_state_dict': model.state_dict(),
                             'opt_state_dict': optimizer.state_dict()}
            if step % 100 == 0:
                print('epoch : {}, global_step : {}, tr_loss: {:.3f}, tr_acc: {:.2%}'.format(
                        epoch + 1, global_step, tr_summary['loss'], tr_summary['acc']))
            

# 모델 저장

In [ ]:
torch.save(model, 'KoBERT_DIS_NER/Human_DIS.pth')

# 모델 평가

In [ ]:
def index_to_ner(sequences, val_len):

    idx_to_ner = {
        1 : "[CLS]", 
        2 : "[SEP]", 
        0 : "O", #[PAD] to 'O'
        3 : "[MASK]",
        4 : "O",
        5 : "B-DIS",
        6 : "I-DIS"
    }
    result = []
    for i in range(val_len):
        result.append(idx_to_ner[sequences[i]])
    return result

In [ ]:
device = torch.device('cuda')
model_config = Config('config.json')
model = torch.load('KoBERT_DIS_NER/Human_PET_DIS2.pth')
model.to(device)
model.eval()
eval_step = 0
sum_f1 = 0
predic_list = []
true_list = []
predic_list2= []
true_list2 = []
for step, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)

    log_likelihood, sequence_of_tags = model(token_ids,valid_length, segment_ids, label)
    # loss: negative log-likelihood
    eval_loss = -1 * log_likelihood

    if (eval_step + 1) % model_config.gradient_accumulation_steps == 0:
        eval_step += 1
        with torch.no_grad():
            sequence_of_tags = torch.tensor(
                sequence_of_tags).to(device)
            eval_acc = (sequence_of_tags == label).float()[label != 0].mean()
        eval_loss_avg = eval_loss / eval_step
        eval_summary = {'loss': eval_loss_avg, 'acc': eval_acc}

        predic_list.append(index_to_ner(sequence_of_tags[0].cpu().numpy(), valid_length[0].cpu().numpy()))
        true_list.append(index_to_ner(label[0].cpu().numpy(), valid_length[0].cpu().numpy()))
        
        if eval_step % 10 == 0:
            print('eval_step : {}, evl_loss: {:.3f}, evl_acc: {:.2f}'.format(
                eval_step, eval_summary['loss'], eval_summary['acc']))

## F1 스코어 평가

In [ ]:
f1_score(true_list, predic_list)

# 사용자 입력 문장 모델 평가

In [ ]:
bertmodel, _ = get_pytorch_kobert_model() # KoBERT 모델 불러오기
vocab = nlp.vocab.BERTVocab.from_sentencepiece('vocab.model', padding_token='[PAD]')
tokenizer = "vocab.model" #생성한 vocab.model 이용
tok = BERTSPTokenizer(tokenizer, vocab, lower=False) # BERTSPTokenizer 생성

In [ ]:
model_config = Config('config.json')

# model
model = torch.load('KoBERT_DIS_NER/Human_PET_DIS.pth')
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

input_text = input('input> ')
print(input_text)
text_sent = []
text_sent.append(input_text)
sent_test = BERTTESTsent(text_sent, tok, 512, True, False)
sent_dataloader = torch.utils.data.DataLoader(sent_test, batch_size=1, num_workers = 5)

for _, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(sent_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids =  segment_ids.long().to(device)
    valid_length =  valid_length
    sequence_of_tags = model(token_ids, valid_length, segment_ids)
    print(sequence_of_tags)
